# History Quiz

## Write a simple class with a few methods for:

- Writing a historical question that has a date as the correct answer
- Getting the correct answer from the LLM
- Getting a human user's best guess at the correct answer
- Checking/reporting the difference between the correct answer and the user's answer

## Loading the libraries

In [1]:
from datetime import datetime
from langchain.output_parsers import DatetimeOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI()
output_parser = DatetimeOutputParser()

## Defining the class

In [2]:
class HistoryQuiz():
    
    def create_history_question(self, topic):
        # this method should output a historical question about the topic that has a date as the correct answer
        # such as "on what date did world war 2 end?"
        system_template = "Create a historical question about the topic {topic} where the correct answer is a year, month and day."
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        system_message_prompt.input_variables
        
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])
        chat_prompt.input_variables
        
        prompt = chat_prompt.format_prompt(topic=topic).to_messages()
        
        question = chat(prompt)
        
        return question
    
    def get_AI_answer(self, question):
        
        template_text = "{request}\n{format_instructions}"
        human_prompt = HumanMessagePromptTemplate.from_template(template_text)
        
        chat_prompt = ChatPromptTemplate.from_messages([human_prompt])
        
        model_request = chat_prompt.format_prompt(request=question, format_instructions=output_parser.get_format_instructions()).to_messages()
        
        result = chat(model_request, temperature=0)
        
        correct_datetime = output_parser.parse(result.content)
        
        return correct_datetime
    
    def get_user_answer(self, question):
        
        year = int(input("Enter year: "))
        month = int(input("Enter month (1-12): "))
        day = int(input("Enter day (1-31): "))
        
        user_datetime = datetime(year, month, day)
        
        try:
            return user_datetime
        except ValueError as e:
            return str(e)
    
    
    def check_user_answer(self, user_answer, ai_answer):
        
        # This method should check the user answer against the AI answer and return the difference betwene them
        time_delta = abs(user_answer - ai_answer)
        days = time_delta.days
              
        answer_difference = f"You were off by {days} days! Correct answer: {ai_answer}"
                                
        return answer_difference
        

## Example usage:

In [3]:
quiz_bot = HistoryQuiz()

### create the question

In [4]:
question = quiz_bot.create_history_question(topic="World War 2")

In [5]:
print(question.content)

On what date did the United States officially enter World War II?


### Create the AI answer

In [6]:
ai_answer = quiz_bot.get_AI_answer(question)

In [7]:
ai_answer

datetime.datetime(1941, 12, 7, 0, 0)

### Get the user answer

In [8]:
user_answer = quiz_bot.get_user_answer(question)

In [9]:
user_answer

datetime.datetime(1941, 4, 23, 0, 0)

### Check the user answer

In [10]:
quiz_bot.check_user_answer(user_answer, ai_answer)

'You were off by 228 days! Correct answer: 1941-12-07 00:00:00'